In [1]:
"""Preprocess Hidden CKD data for model training."""

# Imports
import pandas as pd
from loguru import logger
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import RobustScaler, OrdinalEncoder
from src.config import TABLES_DIR

# Load dataset
df = pd.read_csv(TABLES_DIR / "kfre.csv")
logger.info(f"Loaded dataset with shape: {df.shape}")


# Define preprocessing
num_features = ["age", "egfr", "acr"]
num_transformer = Pipeline([
    ("power_transform", RobustScaler())
])

bin_features = ["female", "dm", "htn", "hf", "cvd"]
bin_categories = [[False, True] for _ in bin_features]
bin_transformer = Pipeline([
    ("ord_enc", OrdinalEncoder(categories=bin_categories))
])

preprocessor = ColumnTransformer(
    transformers=[
        ("num", num_transformer, num_features),
        ("bin", bin_transformer, bin_features),
    ]
)


# Split raw data
X = df[num_features + bin_features]
y = df["esrd"]

X_train_raw, X_test_raw, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)
logger.info(f"Train shape: {X_train_raw.shape}, Test shape: {X_test_raw.shape}")


# Transform train set
logger.info("Fitting preprocessing + SMOTEENN on training data...")
X_train = preprocessor.fit_transform(X_train_raw, y_train) # type: ignore
logger.info(f"Class distribution after resampling:\n{y_train.value_counts()}")

# Transform test set
X_test = preprocessor.transform(X_test_raw)
logger.info(f"Final test shape: {X_test.shape}")

# Extract feature names
feature_names = preprocessor.get_feature_names_out().tolist()
logger.info(f"Extracted {len(feature_names)} feature names.")

# Create and save dataframes to CSV
train_df = pd.DataFrame(X_train, columns=feature_names)
train_df["esrd"] = y_train.values

test_df = pd.DataFrame(X_test, columns=feature_names)
test_df["esrd"] = y_test.values



2025-12-19 17:59:02.541 | INFO     | __main__:<module>:14 - Loaded dataset with shape: (17077, 12)
2025-12-19 17:59:02.550 | INFO     | __main__:<module>:44 - Train shape: (13661, 8), Test shape: (3416, 8)
2025-12-19 17:59:02.550 | INFO     | __main__:<module>:48 - Fitting preprocessing + SMOTEENN on training data...
2025-12-19 17:59:02.558 | INFO     | __main__:<module>:50 - Class distribution after resampling:
esrd
0    13395
1      266
Name: count, dtype: int64
2025-12-19 17:59:02.560 | INFO     | __main__:<module>:54 - Final test shape: (3416, 8)
2025-12-19 17:59:02.561 | INFO     | __main__:<module>:58 - Extracted 8 feature names.


In [2]:
import pandas as pd
import numpy as np
from sksurv.ensemble import RandomSurvivalForest
from sksurv.metrics import concordance_index_censored
from sklearn.inspection import permutation_importance

# -----------------------------
# 1. Load preprocessed train/test sets
# -----------------------------
#train_df = pd.read_csv("train_df.csv")
#test_df = pd.read_csv("test_df.csv")

# Load raw dataset to recover unscaled time
raw_df = pd.read_csv(TABLES_DIR / "kfre.csv")

# -----------------------------
# 2. Prepare survival labels
# -----------------------------
# Align indices with raw time
train_time = raw_df.loc[train_df.index, "time"].values
test_time = raw_df.loc[test_df.index, "time"].values

y_train = np.array([(bool(event), t) for event, t in zip(train_df["esrd"], train_time)],
                   dtype=[("event", bool), ("time", float)])
y_test = np.array([(bool(event), t) for event, t in zip(test_df["esrd"], test_time)],
                  dtype=[("event", bool), ("time", float)])

# Drop target column from features
X_train = train_df.drop(columns=["esrd"])
X_test = test_df.drop(columns=["esrd"])

# -----------------------------
# 3. Train Random Survival Forest
# -----------------------------
rsf = RandomSurvivalForest(
    n_estimators=300,
    min_samples_split=10,
    min_samples_leaf=15,
    max_features="sqrt",
    n_jobs=-1,
    random_state=42
)

rsf.fit(X_train, y_train)

# -----------------------------
# 4. Evaluate performance
# -----------------------------
c_index = concordance_index_censored(y_test["event"], y_test["time"], rsf.predict(X_test))[0]
print(f"Concordance Index (C-index): {c_index:.3f}")


Concordance Index (C-index): 0.937


In [3]:
import numpy as np
import pandas as pd

# Assume rsf, X_test, y_test, test_df, and test_time are already defined

# -----------------------------
# 1. Define horizons (in days)
# -----------------------------
two_years = 2 * 365
five_years = 5 * 365

# -----------------------------
# 2. Predict survival functions
# -----------------------------
# RSF outputs survival curves for each patient
surv_funcs = rsf.predict_survival_function(X_test)

# -----------------------------
# 3. Extract survival probabilities at 2y and 5y
# -----------------------------
pred_2y = []
pred_5y = []

for fn in surv_funcs:
    # fn.x = time points, fn.y = survival probabilities
    # Interpolate survival probability at 2y and 5y
    s2 = np.interp(two_years, fn.x, fn.y)
    s5 = np.interp(five_years, fn.x, fn.y)
    pred_2y.append(1 - s2)  # risk of ESRD within 2 years
    pred_5y.append(1 - s5)  # risk of ESRD within 5 years

# -----------------------------
# 4. Build results dataframe
# -----------------------------
results = test_df.copy()
results["time"] = test_time
results["event"] = y_test["event"]
results["risk_2y"] = pred_2y
results["risk_5y"] = pred_5y

print(results.head())

# -----------------------------
# 5. Quick evaluation
# -----------------------------
# Observed events within 2y and 5y
obs_2y = (results["event"] & (results["time"] <= two_years)).astype(int)
obs_5y = (results["event"] & (results["time"] <= five_years)).astype(int)

print("\nObserved vs Predicted (first 10 patients):")
print(results[["event","time","risk_2y","risk_5y"]].head(10))

# You can also compute calibration or ROC curves at 2y and 5y
from sklearn.metrics import roc_auc_score

auc_2y = roc_auc_score(obs_2y, results["risk_2y"])
auc_5y = roc_auc_score(obs_5y, results["risk_5y"])

print(f"\nAUC at 2 years: {auc_2y:.3f}")
print(f"AUC at 5 years: {auc_5y:.3f}")


   num__age  num__egfr  num__acr  bin__female  bin__dm  bin__htn  bin__hf  \
0 -1.785714   0.538462 -0.371429          1.0      0.0       1.0      0.0   
1  0.214286  -1.153846 -0.300000          0.0      0.0       0.0      0.0   
2 -1.071429   0.230769 -0.085714          1.0      0.0       1.0      0.0   
3  0.142857   0.615385 -0.185714          1.0      1.0       1.0      0.0   
4  0.000000   0.076923  1.342857          0.0      1.0       1.0      0.0   

   bin__cvd  esrd  time  event   risk_2y   risk_5y  
0       0.0     0  3226  False  0.000654  0.001979  
1       1.0     0  2344  False  0.000000  0.008870  
2       0.0     0   699  False  0.000240  0.001656  
3       0.0     0  2933  False  0.000000  0.000069  
4       1.0     0  2771  False  0.001499  0.004348  

Observed vs Predicted (first 10 patients):
   event  time   risk_2y   risk_5y
0  False  3226  0.000654  0.001979
1  False  2344  0.000000  0.008870
2  False   699  0.000240  0.001656
3  False  2933  0.000000  0.000069


In [5]:
scaler = preprocessor.named_transformers_["num"].named_steps["power_transform"]
results[["num__age", "num__egfr", "num__acr"]] = scaler.inverse_transform(results[["num__age", "num__egfr", "num__acr"]].to_numpy())

In [10]:
print(results[results["event"] == True].head(20))

      num__age  num__egfr  num__acr  bin__female  bin__dm  bin__htn  bin__hf  \
29        78.0       33.0       6.3          1.0      0.0       1.0      0.0   
221       57.0       15.0       6.6          0.0      0.0       1.0      0.0   
244       75.0       20.0       9.4          0.0      1.0       1.0      0.0   
355       49.0       38.0       2.2          1.0      1.0       1.0      0.0   
439       76.0       49.0      42.6          0.0      1.0       1.0      0.0   
453       47.0       42.0      20.6          0.0      0.0       1.0      0.0   
464       48.0       39.0      15.1          0.0      0.0       1.0      0.0   
508       86.0       44.0       5.6          0.0      0.0       1.0      1.0   
644       51.0       59.0     395.2          0.0      1.0       1.0      0.0   
689       46.0       25.0     233.9          0.0      0.0       0.0      0.0   
706       74.0       36.0      65.5          0.0      0.0       1.0      0.0   
883       34.0       48.0     110.8     

In [7]:
df[(df["htn"] == True) & (df["dm"] == False) & (df["hf"] == False) & (df["cvd"] == False) & (df["female"] == True) & (df["esrd"] == True)]

,age,female,time,dm,htn,hf,cvd,neph_known,egfr,acr,death,esrd
225,74,True,3306,False,True,False,False,True,42.0,6.8,0,1
240,71,True,77,False,True,False,False,True,10.0,113.1,0,1
1060,47,True,524,False,True,False,False,True,19.0,149.4,0,1
1119,69,True,3059,False,True,False,False,False,43.0,19.0,0,1
1286,58,True,594,False,True,False,False,True,18.0,102.2,0,1
1291,32,True,3034,False,True,False,False,True,44.0,70.3,0,1
1317,59,True,3289,False,True,False,False,True,28.0,107.1,0,1
1765,83,True,1576,False,True,False,False,False,38.0,0.7,0,1
1776,45,True,1535,False,True,False,False,False,27.0,225.3,0,1
2102,74,True,1450,False,True,False,False,False,45.0,10.2,0,1


In [24]:
import math

def kfre_risk(age, female, egfr, acr_mgmmol, years=2):
    """
    Calculate the Kidney Failure Risk Equation (KFRE) 2-year or 5-year risk.
    ACR is provided in mg/mmol (common in UK/Europe).
    
    Parameters:
        age (float): Age in years
        sex (str): 'male' or 'female'
        egfr (float): Estimated GFR (mL/min/1.73m²)
        acr_mgmmol (float): Urine ACR in mg/mmol
        years (int): 2 or 5
        
    Returns:
        float: Risk percentage (0–100)
    """

    # Convert mg/mmol → mg/g
    acr_mgg = acr_mgmmol * 8.84

    # Encode sex: male = 1, female = 0
    male = 0 if female == 1 else 1

    # Linear predictor X (4-variable KFRE)
    X = (0.220 * age) + (0.246 * male) + (0.451 * egfr) + (0.556 * acr_mgg) - 1.957

    # Base risk
    risk = 1 - math.exp(-math.exp(X))
    risk_percent = risk * 100

    # Approximate 5-year scaling
    if years == 5:
        risk_percent = 1 - (1 - risk_percent/100)**2.5
        risk_percent *= 100

    return max(0, min(risk_percent, 100))


In [30]:
kfre_risk(age=24, female=0, egfr=90, acr_mgmmol=2, years=5)

100.0

In [29]:
import math

def kfre_risk(age, female, egfr, acr_mgmmol, years=2):
    """
    Numerically stable KFRE calculation.
    """

    # Convert mg/mmol → mg/g
    acr_mgg = acr_mgmmol * 8.84

    # Encode sex: male = 1, female = 0
    male = 0 if female == 1 else 1

    # Linear predictor
    X = (0.220 * age) + (0.246 * male) + (0.451 * egfr) + (0.556 * acr_mgg) - 1.957

    # ---- Numerically stable risk calculation ----
    if X > 20:
        # exp(X) is astronomically large → risk ~ 100%
        risk = 1.0
    elif X < -20:
        # exp(X) is basically zero → risk ~ 0%
        risk = 1 - math.exp(-1e-9)
    else:
        # Safe region
        risk = 1 - math.exp(-math.exp(X))

    risk_percent = risk * 100

    # 5-year scaling
    if years == 5:
        risk_percent = 1 - (1 - risk_percent/100)**2.5
        risk_percent *= 100

    return max(0, min(risk_percent, 100))


In [22]:
results["kfre_2y"] = results.apply(
	lambda row: kfre_risk(
		age=row["num__age"],
		female=row["bin__female"],
		egfr=row["num__egfr"],
		acr_mgmmol=row["num__acr"],
		years=2,
	),
	axis=1,
)

In [23]:
results

,num__age,num__egfr,num__acr,bin__female,bin__dm,bin__htn,bin__hf,bin__cvd,esrd,time,event,risk_2y,risk_5y,kfre_2y
0,52.0,58.0,1.4,1.0,0.0,1.0,0.0,0.0,0,3226,False,0.000654,0.001979,100.0
1,80.0,36.0,1.9,0.0,0.0,0.0,0.0,1.0,0,2344,False,0.000000,0.008870,100.0
2,62.0,54.0,3.4,1.0,0.0,1.0,0.0,0.0,0,699,False,0.000240,0.001656,100.0
3,79.0,59.0,2.7,1.0,1.0,1.0,0.0,0.0,0,2933,False,0.000000,0.000069,100.0
4,77.0,52.0,13.4,0.0,1.0,1.0,0.0,1.0,0,2771,False,0.001499,0.004348,100.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3411,73.0,55.0,17.6,1.0,1.0,0.0,1.0,1.0,0,2541,False,0.000081,0.001182,100.0
3412,71.0,58.0,1.2,1.0,1.0,1.0,0.0,1.0,0,3294,False,0.000046,0.000255,100.0
3413,86.0,36.0,9.9,0.0,0.0,0.0,0.0,0.0,0,1385,False,0.000267,0.004629,100.0
3414,68.0,38.0,483.5,0.0,1.0,1.0,0.0,0.0,1,606,True,0.065627,0.188974,100.0
